# Get started with the Sampler primitive

Learn how to set up and use the Sampler primitive program.


## Overview

The Sampler primitive lets you more accurately contextualize counts. It takes a user circuit as an input and generates an error-mitigated readout of quasiprobabilities. This enables you to more efficiently evaluate the possibility of multiple relevant data points in the context of destructive interference.  


## Prepare the environment

1. Follow the steps in the [getting started guide](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/getting_started.html) to get your Quantum Service instance ready to use.
2. You'll need at least one circuit to submit to the program. Our examples all have circuits in them, but if you want to submit your own circuit, you can use Qiskit to create one. To learn how to create circuits by using Qiskit, see the [Circuit basics tutorial](https://qiskit.org/documentation/tutorials/circuits/01_circuit_basics.html).


## Start a session

With Qiskit Runtime primitives, we introduce the concept of a session or a factory that allows you to define a job as a collection of iterative calls to the quantum computer. When you start a session, it caches the data you send so it doesn't have to be transmitted to the Quantum Datacenter on each iteration.


### Specify program inputs

The Sampler takes in the following arguments:
- **circuits**: a list of (parameterized) circuits that you want to investigate.
- **parameters**: a list of parameters for the parameterized circuits. It should be omitted if the circuits provided are not parameterized.
- **skip_transpilation**: circuit transpilation is skipped if set to `True`. Default value is `False`.
- **service**: the `QiskitRuntimeService` instance to run the program on. If not specified, the default saved account for `QiskitRuntimeServie` is initialized.
- **options**: Runtime options dictionary that control the execution environment.
  - **backend**: The backend to run on. This option is required if you are running on [IBM Quantum](https://quantum-computing.ibm.com/). However, if you are running on [IBM Cloud](https://cloud.ibm.com/quantum), you can choose not to specify the backend, in which case the least busy backend is used.
  
You can find more details in [the `Sampler` API reference](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Sampler.html).

Example:

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit

service = QiskitRuntimeService()

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

## Write to & read from a session

Running a job and returning the results are done by writing to and reading from the session. The session closes when the code exits the `with` block.


### Run the job & print results

Run the job, specifying your previously defined inputs and options. In this simple example, there is only one circuit and it does not have parameters.

In each call, you will use `circuit_indices` to specify which circuit to run and, if applicable,  `parameter_values` specifies which parameter to use with the specified circuit.

In this example, we specified one circuit, `circuits=bell`, and we asked for the result for running the first (and only) circuit: `circuit_indices=[0]`.

As you will see in later examples, if we had specified multiple circuits, such as `circuits=[pqc, pqc2]`, you could specify `circuit_indices=[1]` to run the `pqc2` circuit.

In [2]:
# executes a Bell circuit
with Sampler(circuits=bell, service=service, options={ "backend": "ibmq_qasm_simulator" }) as sampler:
    result = sampler(circuit_indices=[0], shots=1024)
    print(result)

SamplerResult(quasi_dists=[{'00': 0.5078125, '11': 0.4921875}], metadata=[{'header_metadata': None, 'shots': 1024}])


## Multiple circuit example

In this example, we specify three circuits, but they have no parameters:

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit

service = QiskitRuntimeService()

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# executes three Bell circuits
with Sampler(circuits=[bell]*3, service=service, options={ "backend": "ibmq_qasm_simulator" }) as sampler:
    result = sampler(circuit_indices=[0, 1, 2])
    print(result)

SamplerResult(quasi_dists=[{'00': 0.478515625, '11': 0.521484375}, {'00': 0.533203125, '11': 0.466796875}, {'00': 0.4853515625, '11': 0.5146484375}], metadata=[{'header_metadata': None, 'shots': 1024}, {'header_metadata': None, 'shots': 1024}, {'header_metadata': None, 'shots': 1024}])


## Multiple parameterized circuits example

In this example, we run multiple parameterized circuits. When it is run, this line `result = sampler(circuit_indices=[0, 0, 1], parameter_values=[theta1, theta2, theta3])` specifies which parameter to send to each circuit.  

In our example, the parameter labeled `theta` is sent to the first circuit, `theta2` is sent to the first circuit, and `theta3` is sent to the second circuit.

In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes

service = QiskitRuntimeService()

# parameterized circuit
pqc = RealAmplitudes(num_qubits=2, reps=2)
pqc.measure_all()
pqc2 = RealAmplitudes(num_qubits=2, reps=3)
pqc2.measure_all()

theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [1, 2, 3, 4, 5, 6]
theta3 = [0, 1, 2, 3, 4, 5, 6, 7]

with Sampler(circuits=[pqc, pqc2], service=service, options={ "backend": "ibmq_qasm_simulator" }) as sampler:
    result = sampler(circuit_indices=[0, 0, 1], parameter_values=[theta1, theta2, theta3])
    print(result)

SamplerResult(quasi_dists=[{'00': 0.1259765625, '10': 0.1005859375, '11': 0.408203125, '01': 0.365234375}, {'01': 0.025390625, '11': 0.3056640625, '00': 0.0693359375, '10': 0.599609375}, {'11': 0.03515625, '01': 0.67578125, '10': 0.0888671875, '00': 0.2001953125}], metadata=[{'header_metadata': None, 'shots': 1024}, {'header_metadata': None, 'shots': 1024}, {'header_metadata': None, 'shots': 1024}])


### Result

The results align with the parameter - circuit pairs specified previously.  For example, the first result (`{'11': 0.42578125, '00': 0.14453125, '10': 0.0888671875, '01': 0.3408203125}`) is the output of the parameter labeled `theta` being sent to the first circuit.